# Houses and Empty Lots for Sale in New Brunswick (as of June 12, 2020)

In this project, I scraped data from this [website](https://www.point2homes.com/CA/Real-Estate-Listings/NB.html) which lists houses and 
empty lots for sale. The selling price is listed along with the lot size. For houses, the number of bedrooms, bathrooms, the house size the house type are also given.

## 1. Training and Validation Sets

In order to implement any machine learning algorithms on the data, we must split the dataset into a training set and validation set. For this analysis, 85% of the data will be used for training, while 15% will be used for validation. 

In [1]:
# load libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import re
from tqdm.notebook import tqdm

# load dataset
df = pd.read_csv('houseprice_withpostalcodes.csv')

# look at first 5 rows
df.head(5)

,Address,Beds,Baths,House Size (sqft),Lot Size (ac),Type,Price,Postal Code
0,"10 Robin Dr, Fredericton, New Brunswick, E3C 1K6",5.0,2.0,1600.0,0.177,Residential,259900,E3C 1K6
1,"03-2 Glebe Rd, Saint Andrews, New Brunswick",NaN,NaN,NaN,1.140,Residential,11400,E5B2Z5
2,"62 Parkin Street, Salisbury, New Brunswick, E4...",3.0,2.0,3790.0,0.850,Residential,549900,E4J 2N3
3,"14 Murray Lane, St. Andrews, New Brunswick",4.0,2.0,2200.0,1.150,Residential,449500,E3L4M6
4,"140 Orleans St., Dieppe, New Brunswick, E1A 1W9",4.0,3.0,2808.0,0.124,Residential,236900,E1A 1W9


Believe it or not, the website lists duplicate addresses.

In [2]:
print('Out of', len(df.Address), 'addresses,', len(df.Address.unique()),'addresses are unique.')

Out of 720 addresses, 643 addresses are unique.


and so we remove these duplicates.

In [3]:
df.drop_duplicates(subset='Address',keep='first',inplace=True,ignore_index=True)

df.head(5)

,Address,Beds,Baths,House Size (sqft),Lot Size (ac),Type,Price,Postal Code
0,"10 Robin Dr, Fredericton, New Brunswick, E3C 1K6",5.0,2.0,1600.0,0.177,Residential,259900,E3C 1K6
1,"03-2 Glebe Rd, Saint Andrews, New Brunswick",NaN,NaN,NaN,1.140,Residential,11400,E5B2Z5
2,"62 Parkin Street, Salisbury, New Brunswick, E4...",3.0,2.0,3790.0,0.850,Residential,549900,E4J 2N3
3,"14 Murray Lane, St. Andrews, New Brunswick",4.0,2.0,2200.0,1.150,Residential,449500,E3L4M6
4,"140 Orleans St., Dieppe, New Brunswick, E1A 1W9",4.0,3.0,2808.0,0.124,Residential,236900,E1A 1W9


In [4]:
# check new length
len(df['Address'])

643

Before we split the data, we have to check first if there is a class imbalance on the type of house (or land).

In [5]:
# load data visualization library
from plotnine import *

# check unique values for type
df['Type'].unique()

array(['            Residential        ',
       '            Single Family        ',
       '            Vacant Land        ',
       '            Multi-family        ',
       '            Lots and Land        ',
       '            Lots And Land        ', '            Other        ',
       '            MultiFamily        ',
       '            Condominium        ',
       '            Farm And Agriculture        '], dtype=object)

Multi-family and MultiFamily are essentially the same. There are also two entries of Lots And Land. We can easily fix this.

In [6]:
for index, typ in tqdm(df['Type'].items(), total=len(df['Type'])):
    typ = re.search('[A-Z].*[a-z]',typ)
    typ = typ.group(0)
    
    if typ == 'Multi-family' or typ == 'MultiFamily':
        df.loc[index,'Type'] = 'Multifamily'
    elif typ == 'Lots And Land' or typ == 'Lots and Land':
        df.loc[index,'Type'] = 'Lots And Land'
    else:
        df.loc[index,'Type'] = typ

df.head(5)

,Address,Beds,Baths,House Size (sqft),Lot Size (ac),Type,Price,Postal Code
0,"10 Robin Dr, Fredericton, New Brunswick, E3C 1K6",5.0,2.0,1600.0,0.177,Residential,259900,E3C 1K6
1,"03-2 Glebe Rd, Saint Andrews, New Brunswick",NaN,NaN,NaN,1.140,Residential,11400,E5B2Z5
2,"62 Parkin Street, Salisbury, New Brunswick, E4...",3.0,2.0,3790.0,0.850,Residential,549900,E4J 2N3
3,"14 Murray Lane, St. Andrews, New Brunswick",4.0,2.0,2200.0,1.150,Residential,449500,E3L4M6
4,"140 Orleans St., Dieppe, New Brunswick, E1A 1W9",4.0,3.0,2808.0,0.124,Residential,236900,E1A 1W9


In [7]:
# check counts of df['Type']
type_counts = df['Type'].value_counts()
type_counts

Single Family           246
Residential             210
Lots And Land           107
Vacant Land              51
Multifamily              18
Other                     6
Condominium               3
Farm And Agriculture      2
Name: Type, dtype: int64

There are certainly class imbalances. Only two farm and agriculture lands and 277 single family homes. We need to sample the validation accordingly to represent the same proportions in both the training and validation sets. We may as well combine **Lots And Land** and **Vacant Land** into Empty Land, and combine **Multifamily** with **Single Family** to produce **Family**, and since there are only a few instances of **Other**, **Condominium** and **Farm and Agriculture** types, we'll just omit them.

In [8]:
df = df[(df.Type != 'Condominium') & (df.Type != 'Farm And Agriculture') & (df.Type != 'Other')]
df.Type.unique()

array(['Residential', 'Single Family', 'Vacant Land', 'Multifamily',
       'Lots And Land'], dtype=object)

In [9]:
# check new length
len(df['Type'])

632

In [11]:
for index, typ in tqdm(df['Type'].items(), total=len(df['Type'])):
    if typ == 'Multifamily' or typ == 'Single Family':
        df.loc[index,'Type'] = 'Family'
    elif typ == 'Lots And Land' or typ == 'Vacant Land':
        df.loc[index,'Type'] = 'Empty Land'

df.head(5)

,Address,Beds,Baths,House Size (sqft),Lot Size (ac),Type,Price,Postal Code
0,"10 Robin Dr, Fredericton, New Brunswick, E3C 1K6",5.0,2.0,1600.0,0.177,Residential,259900,E3C 1K6
1,"03-2 Glebe Rd, Saint Andrews, New Brunswick",NaN,NaN,NaN,1.140,Residential,11400,E5B2Z5
2,"62 Parkin Street, Salisbury, New Brunswick, E4...",3.0,2.0,3790.0,0.850,Residential,549900,E4J 2N3
3,"14 Murray Lane, St. Andrews, New Brunswick",4.0,2.0,2200.0,1.150,Residential,449500,E3L4M6
4,"140 Orleans St., Dieppe, New Brunswick, E1A 1W9",4.0,3.0,2808.0,0.124,Residential,236900,E1A 1W9


In [14]:
# we sample the dataset with the weights representing type_counts
y = df['Price']
X = df.drop('Price',axis=1)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.15, stratify = X['Type'].values)

X_train.head(5)

,Address,Beds,Baths,House Size (sqft),Lot Size (ac),Type,Postal Code
280,"254 Ledge Road, Crocker Hill, New Brunswick",3.0,2.0,NaN,NaN,Residential,E3L3Z3
84,"53 Evergreen Street, Fredericton, New Brunswic...",3.0,2.0,1152.0,NaN,Single Family,E3B6N1
326,"126 King Street, Saint Andrews, New Brunswick",5.0,3.0,3242.0,NaN,Residential,E5B1Y6
109,"178 C K Justason Lane, Pennfield, New Brunswick",3.0,3.0,NaN,NaN,Residential,E5H1S2
365,"124 Chamcook Lake #1 Road, Chamcook, New Bruns...",NaN,NaN,NaN,NaN,Lots And Land,E5B3C3


In [29]:
# combine predictors and response
train = pd.concat([X_train, y_train.to_frame()], axis=1)
val = pd.concat([X_val, y_val.to_frame()], axis=1)

train.reset_index(drop=True,inplace=True)
val.reset_index(drop=True,inplace=True)

train.head(5)

# save training and validation to csv
train.to_csv('train.csv',index=False)
val.to_csv('val.csv',index=False)

It would not make sense if both bedrooms and bathrooms contribute individually to the selling price. We can just make a column called **Rooms** which is simply the sum of the number of the bedrooms and bathrooms.

In [ ]:
df['Rooms'] = df['Beds'] + df['Baths']
df.head(5)

Now we check the type of house (or land).

In [ ]:
df['Type'].unique()

Multi-family and MultiFamily are essentially the same. There are also two entries of Lots And Land. We can easily fix this.

In [ ]:
import re
from tqdm.notebook import tqdm

for index, typ in tqdm(df['Type'].items(), total=len(df['Type'])):
    typ = re.search('[A-Z].*[a-z]',typ)
    typ = typ.group(0)
    
    if typ == 'Multi-family' or typ == 'MultiFamily':
        df['Type'][index] = 'Multifamily'
    else:
        df['Type'][index] = typ

df.head(5)

In [ ]:
# Drop beds, baths and address
df.drop(['Address','Beds','Baths'],axis=1,inplace=True)
df.head(5)

We now check null values.

In [ ]:
for column in df.columns:
    print(column,' has', df[column].isna().sum(), ' null values.')

Typically if rooms is nan, then it is usually a piece of land so we just let the rooms be zero.

In [ ]:
df['Rooms'].fillna(0, inplace=True)
df.head(5)

As for the house size and lot size, we can solve the null values if we could use the lot size instead. If there is no data on the lot size, we can use the house size, if there is any.

In [ ]:
# just make a column called size. It is lot size if lot size is present. Otherwise it is house size.
df['Size'] = np.nan

for ind in tqdm(range(len(df['Lot Size (ac)']))):
    if ~np.isnan(df.iloc[ind,1]):
        df.iloc[ind,6] = df.iloc[ind,1]
    elif np.isnan(df.iloc[ind,1]) and ~np.isnan(df.iloc[ind,0]):
        # convert sqft to ac
        acre = df.iloc[ind,0]*2.29*10**(-5)
        df.iloc[ind,6] = acre

df.head(5)

In [ ]:
# Now we check the null values for size
print('Size has', df['Size'].isna().sum(), ' null values.')

265 is not bad, compared to 371 and 496 earlier. Perhaps the best thing to do right now is to impute the size with the mean of its **corresponding house type**.

In [ ]:
for index, size in tqdm(df['Size'].items(), total=len(df['Size'])):
    if np.isnan(size):
        typ = df.iloc[index,2]
        df.iloc[index,6] = df.groupby('Type')['Size'].mean()[typ]

df.head(20)